In [1]:
import os
import psycopg2
import psycopg2.extensions
import logging
import pandas as pd

/home/pavel/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
logger.info("Создаём подключёние к Postgres")
params = {
    "host": '/var/run/postgresql',
    "port": '5432',
    "user": 'pavel'
}
conn = psycopg2.connect(**params)

2018-08-26 22:51:11,251 : INFO : Создаём подключёние к Postgres


In [4]:
psycopg2.extensions.register_type(
    psycopg2.extensions.UNICODE,
    conn
)
conn.set_isolation_level(
    psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
)
cursor = conn.cursor()

In [13]:
user_item_query_config = {
    "MIN_USERS_FOR_ITEM": 10,
    "MIN_ITEMS_FOR_USER": 3,
    "MAX_ITEMS_FOR_USER": 50,
    "MAX_ROW_NUMBER": 100000
}
sql_str = (
        """
          SELECT * FROM sales limit 100000;
        """ % user_item_query_config
)

In [28]:
cursor.execute(sql_str)
ui_data = [a for a in cursor.fetchall()]
conn.commit()
logger.info("Данные по продажам загружены из Postgres")

2018-08-26 23:06:54,494 : INFO : Данные по продажам загружены из Postgres


In [29]:
sales = pd.DataFrame(ui_data)

In [30]:
sales.columns = ['OPER_DATE', 'CODE_OFFICE', 'TP', 'DOC_NUMBER', 'ITEM_ARTICLE', 'QTY', 'AMOUNT_AUTO_DISCOUNT', 'AMOUNT_RUR', 'PAYMENT_FORM', 'KEY_DISCOUNT']

In [31]:
sales.head()

,OPER_DATE,CODE_OFFICE,TP,DOC_NUMBER,ITEM_ARTICLE,QTY,AMOUNT_AUTO_DISCOUNT,AMOUNT_RUR,PAYMENT_FORM,KEY_DISCOUNT
0,2017-07-09 17:02:00,313002,I,1313002006124,PB,1,None,200,Наличная,1.0
1,2017-07-09 17:03:00,277254,I,1277254015887,PB,1,None,500,Платежная карта,1.0
2,2017-07-09 17:03:00,363028,I,1363028008070,K3,1,None,30297,Наличная,NaN
3,2017-07-09 17:03:00,736006,I,1736006009235,0905022730,1,None,240,Наличная,1.0
4,2017-07-02 14:33:00,706002,I,1706002005763,PB,1,None,200,Наличная,1.0


In [32]:
user_item_query_config = {
    "MIN_USERS_FOR_ITEM": 10,
    "MIN_ITEMS_FOR_USER": 3,
    "MAX_ITEMS_FOR_USER": 50,
    "MAX_ROW_NUMBER": 100000
}
sql_str = (
        """
          SELECT * FROM items limit 100000;
        """ % user_item_query_config
)

In [33]:
cursor.execute(sql_str)
ui_data = [a for a in cursor.fetchall()]
conn.commit()
logger.info("Данные по артикулам загружены из Postgres")

2018-08-26 23:08:47,139 : INFO : Данные по артикулам загружены из Postgres


In [34]:
items = pd.DataFrame(ui_data)

In [37]:
items.columns = ['ITEM_ARTICLE','ITEM_NAME','ITEM_CATEGORY', 'ITEM_SUBCATEGORY','ITEM_MARK','VIEW_ITEM','MOD_NO_COLOR']

In [38]:
items.head()

,ITEM_ARTICLE,ITEM_NAME,ITEM_CATEGORY,ITEM_SUBCATEGORY,ITEM_MARK,VIEW_ITEM,MOD_NO_COLOR
0,0105001240,Чех.UCHUN7BB беж-син. Untamo,Accessories,Чехол,Untamo,Товары без серий,Чехол UCHUN7BB беж-син. Untamo
1,0105001250,Чех.UCHUN7GL серо-салат.Untamo,Accessories,Чехол,Untamo,Товары без серий,Чехол UCHUN7GL серо-салат.Untamo
2,0105001260,Чех. UACBTABBL черн. Untamo,Accessories,Чехол,Untamo,Товары без серий,Чехол UACBTABBL черн. Untamo
3,0105001270,Чех.CPC_AP_I5_BLACK APEXTO,Accessories,Чехол,Lone,Товары без серий,Чех.CPC_AP_I5_BLACK APEXTO
4,0201000490,Р/т Motorola V100 GSM,Phones,FEATUREPHONES,Motorola,Товары без серий,Motorola V100


In [39]:
user_item_query_config = {
    "MIN_USERS_FOR_ITEM": 10,
    "MIN_ITEMS_FOR_USER": 3,
    "MAX_ITEMS_FOR_USER": 50,
    "MAX_ROW_NUMBER": 100000
}
sql_str = (
        """
          SELECT * FROM shops limit 100000;
        """ % user_item_query_config
)

In [40]:
cursor.execute(sql_str)
ui_data = [a for a in cursor.fetchall()]
conn.commit()
logger.info("Данные по справочнику ТТ из Postgres")

2018-08-26 23:13:31,857 : INFO : Данные по справочнику ТТ из Postgres


In [41]:
shops = pd.DataFrame(ui_data)

In [43]:
shops.columns = ['MARKET_CODE', 'SHOP', 'CODE_POINTS_SALE', 'CODE_OFFICE', 'SHEDULE_NAME', 'TIME_ZONE']

In [44]:
shops.head()

,MARKET_CODE,SHOP,CODE_POINTS_SALE,CODE_OFFICE,SHEDULE_NAME,TIME_ZONE
0,SPB,ЦКЗ (Измайловский) Внешние сотрудники,41816,478161,None,None
1,VIP,"Москва, Грекова д.8",056000,277004,График ежедневно 8:00 - 22:00,GMT+03:00
2,YAK,"Республика Саха (Якутия), п. Хандыга,ул, П. Ал...",7M42000,114042,None,None
3,CHT,672038 Чита г Шилова ул. 100 стр. 2,7K71000,175029,None,None
4,BUR,670033 Улан-Удэ г Жердева ул 104 б,7K69000,432083,None,GMT+08:00


In [47]:
df = pd.merge(sales, items, how = 'left', on = 'ITEM_ARTICLE')

In [52]:
df2 = df.loc[:, ['OPER_DATE', 'DOC_NUMBER', 'ITEM_ARTICLE', 'AMOUNT_RUR']]

In [68]:
df3 = pd.DataFrame(df2.groupby('DOC_NUMBER')['AMOUNT_RUR'].sum())

In [69]:
df4 = pd.merge(df2, df3, how = 'left', on = 'DOC_NUMBER')

In [73]:
df4.to_csv('agg_date_from_postgres.csv', sep=',', header=True)